<a target="_blank" href="https://colab.research.google.com/github/yandex-research/rtdl-num-embeddings/blob/main/package/example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

---

**See also** [RTDL](https://github.com/yandex-research/rtdl)
-- **other projects on tabular deep learning**.

---

- This notebook provides a usage example of the
  [rtdl_num_embeddings](https://github.com/yandex-research/rtdl-num-embeddings)
  package.
- Hyperparameters are not tuned and may be suboptimal.

In [1]:
# If the notebook fails because of these additional packages,
# try install specific versions:
# %pip install delu==0.0.25
# %pip install rtdl_revisiting_models==0.0.2

%pip install delu
%pip install rtdl_revisiting_models

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# ruff: noqa: E402
import math
import warnings
from typing import Literal, Optional

warnings.simplefilter('ignore')
import delu  # Deep Learning Utilities: https://github.com/Yura52/delu
import numpy as np
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from tqdm.std import tqdm

warnings.resetwarnings()

import rtdl_revisiting_models
import rtdl_num_embeddings

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Set random seeds in all libraries.
delu.random.seed(0)

0

## Dataset

In [4]:
# >>> Dataset.
TaskType = Literal['regression', 'binclass', 'multiclass']

task_type: TaskType = 'regression'
n_classes = None
dataset = sklearn.datasets.fetch_california_housing()
X_cont: np.ndarray = dataset['data']
Y: np.ndarray = dataset['target']

# NOTE: uncomment to solve a classification task.
# n_classes = 2
# assert n_classes >= 2
# task_type: TaskType = 'binclass' if n_classes == 2 else 'multiclass'
# X_cont, Y = sklearn.datasets.make_classification(
#     n_samples=20000,
#     n_features=8,
#     n_classes=n_classes,
#     n_informative=3,
#     n_redundant=2,
# )

# >>> Continuous features.
X_cont: np.ndarray = X_cont.astype(np.float32)
n_cont_features = X_cont.shape[1]

# >>> Categorical features.
# NOTE: the above datasets do not have categorical features, but,
# for the demonstration purposes, it is possible to generate them.
cat_cardinalities = [
    # NOTE: uncomment the two lines below to add two categorical features.
    # 4,  # Allowed values: [0, 1, 2, 3].
    # 7,  # Allowed values: [0, 1, 2, 3, 4, 5, 6].
]
X_cat = (
    np.column_stack(
        [np.random.randint(0, c, (len(X_cont),)) for c in cat_cardinalities]
    )
    if cat_cardinalities
    else None
)

# >>> Labels.
# Regression labels must be represented by float32.
if task_type == 'regression':
    Y = Y.astype(np.float32)
else:
    assert n_classes is not None
    Y = Y.astype(np.int64)
    assert set(Y.tolist()) == set(
        range(n_classes)
    ), 'Classification labels must form the range [0, 1, ..., n_classes - 1]'

# >>> Split the dataset.
all_idx = np.arange(len(Y))
trainval_idx, test_idx = sklearn.model_selection.train_test_split(
    all_idx, train_size=0.8
)
train_idx, val_idx = sklearn.model_selection.train_test_split(
    trainval_idx, train_size=0.8
)
data_numpy = {
    'train': {'x_cont': X_cont[train_idx], 'y': Y[train_idx]},
    'val': {'x_cont': X_cont[val_idx], 'y': Y[val_idx]},
    'test': {'x_cont': X_cont[test_idx], 'y': Y[test_idx]},
}
if X_cat is not None:
    data_numpy['train']['x_cat'] = X_cat[train_idx]
    data_numpy['val']['x_cat'] = X_cat[val_idx]
    data_numpy['test']['x_cat'] = X_cat[test_idx]

## Preprocessing

In [5]:
# >>> Feature preprocessing.
# NOTE
# The choice between preprocessing strategies depends on a task and a model.

# (A) Simple preprocessing strategy.
# preprocessing = sklearn.preprocessing.StandardScaler().fit(
#     data_numpy['train']['x_cont']
# )

# (B) Fancy preprocessing strategy.
# The noise is added to improve the output of QuantileTransformer in some cases.
X_cont_train_numpy = data_numpy['train']['x_cont']
noise = (
    np.random.default_rng(0)
    .normal(0.0, 1e-5, X_cont_train_numpy.shape)
    .astype(X_cont_train_numpy.dtype)
)
preprocessing = sklearn.preprocessing.QuantileTransformer(
    n_quantiles=max(min(len(train_idx) // 30, 1000), 10),
    output_distribution='normal',
    subsample=10**9,
).fit(X_cont_train_numpy + noise)
del X_cont_train_numpy

for part in data_numpy:
    data_numpy[part]['x_cont'] = preprocessing.transform(data_numpy[part]['x_cont'])

# >>> Label preprocessing.
if task_type == 'regression':
    Y_mean = data_numpy['train']['y'].mean().item()
    Y_std = data_numpy['train']['y'].std().item()
    for part in data_numpy:
        data_numpy[part]['y'] = (data_numpy[part]['y'] - Y_mean) / Y_std

# >>> Convert data to tensors.
data = {
    part: {k: torch.as_tensor(v, device=device) for k, v in data_numpy[part].items()}
    for part in data_numpy
}

if task_type != 'multiclass':
    # Required by F.binary_cross_entropy_with_logits
    for part in data:
        data[part]['y'] = data[part]['y'].float()

## Model

In [12]:
class Model(nn.Module):
    def __init__(
        self,
        n_cont_features: int,
        cat_cardinalities: list[int],
        bins: Optional[list[Tensor]],
        mlp_kwargs: dict,
    ) -> None:
        super().__init__()
        self.cat_cardinalities = cat_cardinalities
        # The total representation size for categorical features
        # == the sum of one-hot representation sizes
        # == the sum of the numbers of distinct values of all features.
        d_cat = sum(cat_cardinalities)

        # Choose any of the embeddings below.

        # d_embedding = 24
        # self.cont_embeddings = rtdl_num_embeddings.PeriodicEmbeddings(
        #     n_cont_features, d_embedding, lite=False
        # )
        # d_num = n_cont_features * d_embedding

        # assert bins is not None
        # self.cont_embeddings = rtdl_num_embeddings.PiecewiseLinearEncoding(bins)
        # d_num = sum(len(b) - 1 for b in bins)

        assert bins is not None
        d_embedding = 8
        self.cont_embeddings = rtdl_num_embeddings.PiecewiseLinearEmbeddings(
            bins, d_embedding, activation=False, version='B'
        )
        d_num = n_cont_features * d_embedding

        # d_embedding = 32
        # self.cont_embeddings = rtdl_num_embeddings.LinearReLUEmbeddings(
        #     n_cont_features, d_embedding
        # )
        # d_num = n_cont_features * d_embedding

        self.backbone = rtdl_revisiting_models.MLP(d_in=d_num + d_cat, **mlp_kwargs)

    def forward(self, x_cont: Tensor, x_cat: Optional[Tensor]) -> Tensor:
        x = []

        # Step 1. Embed the continuous features.
        # Flattening is needed for MLP-like models.
        x.append(self.cont_embeddings(x_cont).flatten(1))

        # Step 2. Encode the categorical features using any strategy.
        if x_cat is not None:
            x.extend(
                F.one_hot(column, cardinality)
                for column, cardinality in zip(x_cat.T, self.cat_cardinalities)
            )

        # Step 3. Assemble the vector input for the backbone.
        x = torch.column_stack(x)

        # Step 4. Apply the backbone.
        return self.backbone(x)


# This is needed only for PiecewiseLinearEncoding and PiecewiseLinearEmbeddings.
bins = rtdl_num_embeddings.compute_bins(data['train']['x_cont'])
model = Model(
    n_cont_features,
    cat_cardinalities,
    bins,
    {
        'n_blocks': 2,
        'd_block': 384,
        'dropout': 0.4,
        'd_out': n_classes if task_type == 'multiclass' else 1,
    },
).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

## Training

In [13]:
def apply_model(batch: dict[str, Tensor]) -> Tensor:
    return model(batch['x_cont'], batch.get('x_cat')).squeeze(-1)


loss_fn = (
    F.binary_cross_entropy_with_logits
    if task_type == 'binclass'
    else F.cross_entropy
    if task_type == 'multiclass'
    else F.mse_loss
)


@torch.no_grad()
def evaluate(part: str) -> float:
    model.eval()

    eval_batch_size = 8096
    y_pred = (
        torch.cat(
            [
                apply_model(batch)
                for batch in delu.iter_batches(data[part], eval_batch_size)
            ]
        )
        .cpu()
        .numpy()
    )
    y_true = data[part]['y'].cpu().numpy()

    if task_type == 'binclass':
        y_pred = np.round(scipy.special.expit(y_pred))
        score = sklearn.metrics.accuracy_score(y_true, y_pred)
    elif task_type == 'multiclass':
        y_pred = y_pred.argmax(1)
        score = sklearn.metrics.accuracy_score(y_true, y_pred)
    else:
        assert task_type == 'regression'
        score = -(sklearn.metrics.mean_squared_error(y_true, y_pred) ** 0.5 * Y_std)
    return score  # The higher -- the better.


print(f'Test score before training: {evaluate("test"):.4f}')

Test score before training: -1.1612


In [14]:
# For demonstration purposes (fast training and bad performance),
# one can set smaller values:
# n_epochs = 20
# patience = 2
n_epochs = 1_000_000_000
patience = 16

batch_size = 256
epoch_size = math.ceil(len(train_idx) / batch_size)
timer = delu.tools.Timer()
early_stopping = delu.tools.EarlyStopping(patience, mode='max')
best = {
    'val': -math.inf,
    'test': -math.inf,
    'epoch': -1,
}

print(f'Device: {device.type.upper()}')
print('-' * 88 + '\n')
timer.run()
for epoch in range(n_epochs):
    for batch in tqdm(
        delu.iter_batches(data['train'], batch_size, shuffle=True),
        desc=f'Epoch {epoch}',
        total=epoch_size,
    ):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(apply_model(batch), batch['y'])
        loss.backward()
        optimizer.step()

    val_score = evaluate('val')
    test_score = evaluate('test')
    print(f'(val) {val_score:.4f} (test) {test_score:.4f} [time] {timer}')

    early_stopping.update(val_score)
    if early_stopping.should_stop():
        break

    if val_score > best['val']:
        print('🌸 New best epoch! 🌸')
        best = {'val': val_score, 'test': test_score, 'epoch': epoch}
    print()

print('\n\nResult:')
print(best)

Device: CPU
----------------------------------------------------------------------------------------



Epoch 0: 100%|██████████| 52/52 [00:00<00:00, 246.63it/s]


(val) -0.6263 (test) -0.6354 [time] 0:00:00.225773
🌸 New best epoch! 🌸



Epoch 1: 100%|██████████| 52/52 [00:00<00:00, 256.21it/s]


(val) -0.5990 (test) -0.6039 [time] 0:00:00.442324
🌸 New best epoch! 🌸



...

(val) -0.4735 (test) -0.4786 [time] 0:00:31.673475



Epoch 143: 100%|██████████| 52/52 [00:00<00:00, 247.36it/s]


(val) -0.4653 (test) -0.4702 [time] 0:00:31.897596



Epoch 144: 100%|██████████| 52/52 [00:00<00:00, 243.74it/s]

(val) -0.4689 (test) -0.4718 [time] 0:00:32.124995


Result:
{'val': -0.4610990747680548, 'test': -0.4665925272393832, 'epoch': 128}
